In [1]:
import numpy as np
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
import itertools as it
%matplotlib inline

In [2]:
def routes_jalan(G, routes):
    t = 'travel_time'
    list_jalan = []
    for i in range(len(routes)-1):
        minimum = G[routes[i]][routes[i+1]][0][t]
        print(minimum)
        j = 0
        keys = 0
        for j in range(len(G.get_edge_data(routes[i], routes[i+1]))):
            if G.get_edge_data(routes[i], routes[i+1], j)[t] < minimum:
                minimum = G.get_edge_data(routes[i], routes[i+1], j)[t]
                keys = j
        list_jalan.append(keys)
    
    return list_jalan

In [ ]:
df = nx.to_pandas_edgelist(G, edge_key='ekey').set_index(["source","target","ekey"])
df[attr_name] = df[value].values
nx.set_edge_attrbibutes(G, df[attr_name], name=attr_name)

In [4]:
##kayaknya gaperlu di studi kasus, ini initialize flow. Changing the name to add_edge_flow from updating_mainflow
def add_edge_travel_time(G):
    df = nx.to_pandas_edgelist(G, edge_key='ekey').set_index(["source","target","ekey"])
    df['travel_time'] = df['travel_time_awal'].values
    nx.set_edge_attributes(G, df['travel_time'], name='travel_time')
    return G  

In [6]:
def add_edge_flow(G, flow):
    df = nx.to_pandas_edgelist(G, edge_key='ekey').set_index(["source","target","ekey"])
    df['flow'] = flow.values
    nx.set_edge_attributes(G, df['flow'], name='flow')
    return G  

In [5]:
def updating_auxflow(G,auxflow):
    df = nx.to_pandas_edgelist(G, edge_key='ekey').set_index(["source","target","ekey"])
    df['auxflow'] = auxflow.values
    nx.set_edge_attributes(G, df['auxflow'], name='auxflow')  
    return G  

In [6]:
#This is a link performance function derived from Indonesia's Highway Manual (MKJI 1997).
def lpr(row, alpha=0.15,beta=4):
    if np.isnan(row['flow']):
        return row['travel_time']
    else:
        try:
            return row['travel_time_awal']*(1+alpha*(row['flow']/row['capacity'])**beta)
        except ValueError:
            return row['travel_time']

In [3]:
def updating_travel_time_lpr(G, alpha=0.15,beta=4):
    df = nx.to_pandas_edgelist(G, edge_key='ekey').set_index(["source","target","ekey"])
    df['travel_time'] = df.apply(lambda x: lpr(x, alpha, beta), axis=1).values
    nx.set_edge_attributes(G, df['travel_time'], name='travel_time')  
    return G  

In [7]:
def line_search(row, a, alpha, beta):
    t=row['travel_time_awal']
    x=row['flow']
    y=row['auxflow']
    c=row['capacity']
    return -t*(x-y)*(alpha*(((x+a*(y-x))/c)**beta)+1)

In [8]:
def update(row, alpha):
    return row['flow'] + alpha * (row['auxflow'] - row['flow'])

In [9]:
def bisection(G, xl=0, xr=1, delta=0.0002, alpha=0.15, beta=4):
    n = 0
    df = nx.to_pandas_edgelist(G)
    condition = True
    while condition:
        n += 1
        x = (xl+xr)/2
        if df.apply(lambda row: line_search(row, x, alpha=alpha, beta=beta), axis = 1).sum() <= 0:
            xl = x
        else:
            xr = x
        condition = abs(xr-xl) > 2*delta
        # print('|xl-xr| =', abs(xr-xl),' xl =', xl, ' xr=',xr)
    # print(f"number of iteration in bisection method is {n} with alpha = {(xr+xl)/2}")
    return (xr+xl)/2

# Convev Combinations Algorithm

In [10]:
def CCA(G, source, target, flow, convergence):
    
    #Initialize time_travel based on travel_time_awal (dibikin fungsi baru aja kayaknya)
    add_edge_travel_time(G)

     # Initialize flow, fix it.
    routes = nx.shortest_path(G, source, target, weight='travel_time_awal')
    G = add
    
    condition = True

    while condition == True:
        unique_edge = set(G.edges())
        #update travel time based on Link Performance Function
        for i in unique_edge:
            for j in range(len(G.get_edge_data(i[0],i[1]))):
                attrib = {(i[0], i[1], j): {"travel_time": lpr_idn(G[i[0]][i[1]][j])}}
                nx.set_edge_attributes(G, attrib)

        # calculate auxiliary flow from shortest path from new travel time
        routes = nx.shortest_path(G, source, target, weight='travel_time')
        G = updating_auxflow(G, routes)

        # calculate optimal move size (alpha)
        alpha = bisection(G)
        if alpha < convergence:
            condition = False

        print('alpha =', alpha)

        #updating main flow based on optimal move size.
        flow_baru = df.apply(lambda row: update(row, alpha), axis=1)

        #set edge attribute into new main flow.
        index = 0
        for i in unique_edge:
            for j in range(len(G.get_edge_data(i[0],i[1]))):
                    attrib = {(i[0], i[1], j): {new_flow: flow_baru[index]}}
                    nx.set_edge_attributes(G, attrib)
                    index += 1
                
    return G

In [11]:
# Contoh
G = nx.MultiDiGraph()
key = G.add_edge(1,2, capacity = 2, travel_time_awal=10)
key2 = G.add_edge(1,2, capacity = 4, travel_time_awal=20)
key3 = G.add_edge(1,2, capacity = 3, travel_time_awal=25)
# key4 = G.add_edge(2,3, capacity = 5, travel_time_awal=37)
# key5 = G.add_edge(2,3, capacity = 5, travel_time_awal=15)
flow = 10

H = CCA(G,1,2, 0.01)
nx.to_pandas_edgelist(H)

KeyError: 'travel_time_awal'

In [55]:
# Contoh
G = nx.MultiDiGraph()
key = G.add_edge(1,2, capacity = 2, travel_time_awal=10)
key2 = G.add_edge(1,2, capacity = 4, travel_time_awal=20)
key3 = G.add_edge(1,2, capacity = 3, travel_time_awal=25)
# key4 = G.add_edge(2,3, capacity = 5, travel_time_awal=37)
# key5 = G.add_edge(2,3, capacity = 5, travel_time_awal=15)
df= pd.DataFrame()
add_edge_attributes(G, value='travel_time_awal', attr_name='travel_time')
df = nx.to_pandas_edgelist(G, edge_key='ekey')
display(df)
add_edge_attributes(G, value='travel_time', attr_name='travel_time', method_chain=['apply', 'iloc'], args=['lambda x: lpr(x): axis=1', '[:, 0]'])
df = nx.to_pandas_edgelist(G, edge_key='ekey')
display(df)


,source,target,ekey,travel_time,travel_time_awal,capacity
0,1,2,0,10,10,2
1,1,2,1,20,20,4
2,1,2,2,25,25,3


SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
def foo(string):
    df = df

In [ ]:
# Contoh
G = nx.MultiDiGraph()
key = G.add_edge(1,2, capacity = 2, travel_time_awal=10)
key2 = G.add_edge(1,2, capacity = 4, travel_time_awal=20)
key3 = G.add_edge(1,2, capacity = 3, travel_time_awal=25)
# key4 = G.add_edge(2,3, capacity = 5, travel_time_awal=37)
# key5 = G.add_edge(2,3, capacity = 5, travel_time_awal=15)
df= pd.DataFrame()
add_edge_attributes(G, value='travel_time_awal', attr_name='travel_time')
df = nx.to_pandas_edgelist(G, edge_key='ekey')
display(df)

In [ ]:
df = nx.to_pandas_edgelist(G, edge_key='ekey')
display(df)

,source,target,ekey,travel_time,travel_time_awal,capacity
0,1,2,0,10,10,2
1,1,2,1,20,20,4
2,1,2,2,25,25,3


In [ ]:
display(df[["ekey","travel_time_awal"]])

,ekey,travel_time_awal
0,0,10
1,1,20
2,2,25
